# Zusätzliches Material: Webscraping

In [110]:
# flatten list
all_results_flat = [i for s in all_results for i in s]
all_results_flat

[['https://www.cyclingnews.com/news/lorena-wiebes-leaves-giro-donne-to-focus-on-tour-de-france-femmes-sprints/',
  'Lorena Wiebes leaves Giro Donne to focus on Tour de France Femmes sprints',
  '2023-07-06T14:21:02Z',
  'Kirsten Frattini'],
 ['https://www.cyclingnews.com/news/truck-driver-accused-of-davide-rebellin-hit-and-run-death-to-be-extradited-to-italy/',
  'Truck driver accused of Davide Rebellin hit-and-run death to be extradited to Italy',
  '2023-07-06T12:15:38Z',
  'Barry Ryan'],
 [nan, nan, nan, nan],
 ['https://www.cyclingnews.com/news/doctors-give-balsamo-green-light-to-start-training-after-recovery-from-broken-jaw/',
  'Doctors give Balsamo green light to start training after recovery from broken jaw',
  '2023-07-06T00:44:18Z',
  'Kirsten Frattini'],
 ['https://www.cyclingnews.com/news/jumbo-visma-combine-to-outfox-pogacar-and-uae-team-emirates-in-pyrenees-at-tour-de-france/',
  'Jumbo-Visma combine to outfox Pogacar in Pyrenees at Tour de France',
  '2023-07-05T19:11:14

Diese Liste aus Listen können wir nun in einen DataFrame umwandeln.

In [111]:
# list to DataFrame
all_results_df = pd.DataFrame(all_results_flat, columns = ['url', 'title', 'date', 'author'])

In [112]:
all_results_df.head()

,url,title,date,author
0,https://www.cyclingnews.com/news/lorena-wiebes...,Lorena Wiebes leaves Giro Donne to focus on To...,2023-07-06T14:21:02Z,Kirsten Frattini
1,https://www.cyclingnews.com/news/truck-driver-...,Truck driver accused of Davide Rebellin hit-an...,2023-07-06T12:15:38Z,Barry Ryan
2,NaN,NaN,NaN,NaN
3,https://www.cyclingnews.com/news/doctors-give-...,Doctors give Balsamo green light to start trai...,2023-07-06T00:44:18Z,Kirsten Frattini
4,https://www.cyclingnews.com/news/jumbo-visma-c...,Jumbo-Visma combine to outfox Pogacar in Pyren...,2023-07-05T19:11:14Z,Stephen Farrand


In [113]:
all_results_df.groupby('author').size().sort_values(ascending=False)

author
Stephen Farrand                      31
Alasdair Fotheringham                25
Daniel Ostanek                       25
Barry Ryan                           16
Simone Giuliani                      13
Kirsten Frattini                     13
Lukas Knöfler                        11
Paul Norman                           8
Josh Croxton                          6
Laura Weislo                          6
Amy Jones                             5
Tom Wieckowski                        5
Jackie Tyson                          4
Peter Stuart                          2
James Moultrie                        1
Laura Weislo, Jackie Tyson            1
Cyclingnews                           1
Lyne Lamoureux                        1
Patrick Fletcher                      1
Barry Ryan, Stephen Farrand           1
Barry Ryan, Jackie Tyson              1
Barry Ryan, Alasdair Fotheringham     1
Stephen Farrand, Barry Ryan           1
Will Jones                            1
dtype: int64

Nun haben wir eine Tabelle mit allen Artikeln auf der Webseite. Der Text ist aber noch nicht enthalten, da dieser auf einer eigenen Webseite ist. Wir haben aber alle urls dieser Unterwebseiten. Das heißt wir können mit einem Loop über alle Webseiten loopen und dort den entsprechenden Text extrahieren. hierzu schreiben wir zunächst eine Funktion, die wir dann auf alle webseiten anwenden können.

In [114]:
# scrape text for every article

In [115]:
def get_cn_article_text(url):
    results = requests.get(url)
    
    if results.ok:
        soup = BeautifulSoup(results.content, 'html.parser')
        try:
            
            article_text = " ".join(para.text.replace('\n', '') for para in soup.find('div', {'id': 'article-body'}).find_all('p'))

            # replace \xa0
            article_text = article_text.replace('\xa0', ' ')

            # remove whitespace
            article_text = ' '.join(article_text.split())
            
        except:
            article_text = np.nan
        
        return [url, article_text]
    
    else:
        return None

In [116]:
url = 'https://www.cyclingnews.com/news/tour-de-france-femmes-to-start-in-rotterdam-in-2024/'

In [117]:
get_cn_article_text(url)

['https://www.cyclingnews.com/news/tour-de-france-femmes-to-start-in-rotterdam-in-2024/',
 "The Grand Départ of the Tour de France Femmes avec Zwift will be in Rotterdam in 2024, Wielerflits reported, citing multiple unnamed sources. The news should be made official this week by the Amaury Sport Organisation (ASO) in the lead-up to the start of the Tour de France in Bilbao. After starting with a circuit race in Paris for its first edition in 2022 and then a road stage in Clermont-Ferrand in 2023, the third edition of the race will head outside of France for the first time in its history. Wielerflits reports that the first three days of the stage race will take place outside of France, starting with a time trial in Rotterdam on August 12, one day after the end of the Paris Olympic Games. Tour de France Femmes 2023Annemiek van Vleuten seals Tour de France Femmes victoryYou’ve come a long way, baby - Vital statistics show sea change in women’s cycling Stage 2 will be a road stage from Rot

Diese Funktion wenden wir nun auf alle urls an. Dafür benutzen wir einen for-Loop:

In [118]:
all_article_text = []

for url in all_results_df.url:
    print(url)
    try:
        all_article_text.append(get_cn_article_text(url))
    except:
        print('Error! Next')
        next
    time.sleep(1)

https://www.cyclingnews.com/news/lorena-wiebes-leaves-giro-donne-to-focus-on-tour-de-france-femmes-sprints/
https://www.cyclingnews.com/news/truck-driver-accused-of-davide-rebellin-hit-and-run-death-to-be-extradited-to-italy/
nan
Error! Next
https://www.cyclingnews.com/news/doctors-give-balsamo-green-light-to-start-training-after-recovery-from-broken-jaw/
https://www.cyclingnews.com/news/jumbo-visma-combine-to-outfox-pogacar-and-uae-team-emirates-in-pyrenees-at-tour-de-france/
https://www.cyclingnews.com/news/fabio-jakobsen-fights-the-pain-to-stay-in-tour-de-france-after-sprint-crash/
https://www.cyclingnews.com/news/it-wasnt-planned-jai-hindley-races-into-the-tour-de-france-lead-in-pyrenees/
https://www.cyclingnews.com/news/i-wanted-to-test-pogacar-vingegaard-lays-down-marker-at-tour-de-france/
https://www.cyclingnews.com/news/labous-challenges-for-giro-donne-podium-with-aggressive-team-tactics/
https://www.cyclingnews.com/news/tadej-pogacar-keeps-tour-de-france-pyrenees-time-loss-in-

https://www.cyclingnews.com/news/van-der-poel-philipsen-combine-to-create-powerful-tour-de-france-sprint-train/
https://www.cyclingnews.com/news/annemiek-van-vleuten-recharged-and-back-again-for-giro-ditalia/
https://www.cyclingnews.com/news/lotto-dstny-director-allan-davis-will-not-attend-tour-de-france-following-accusations-of-transgressive-behaviour/
https://www.cyclingnews.com/news/jonas-vingegaard-i-expect-tadej-pogacar-to-attack-early-at-the-tour-de-france/
https://www.cyclingnews.com/news/tadej-pogacar-physically-unsure-of-start-at-tour-de-france-but-super-good-mentally/
https://www.cyclingnews.com/news/no-room-for-sentimentality-at-mark-cavendishs-final-tour-de-france/
nan
Error! Next
https://www.cyclingnews.com/news/thibaut-pinot-i-havent-fully-realised-its-my-last-tour-de-france/
https://www.cyclingnews.com/news/face-masks-and-selfie-bans-return-to-limit-covid-19-in-tour-de-france-peloton/
https://www.cyclingnews.com/news/simon-yates-happy-to-fly-below-radar-at-23-tour-de-fra

https://www.cyclingnews.com/news/wout-van-aert-hopes-for-another-step-forward-ahead-of-tour-de-france/
https://www.cyclingnews.com/news/gino-mader-and-safety-foremost-for-jakobsen-after-second-belgium-tour-win/
https://www.cyclingnews.com/news/alberto-contador-gets-multiple-facial-stitches-after-vuelta-espana-event-crash-in-china/
https://www.cyclingnews.com/news/marlen-reusser-its-hard-to-keep-the-focus-on-the-sport/
https://www.cyclingnews.com/news/german-truck-driver-arrested-over-hit-and-run-death-of-davide-rebellin/
https://www.cyclingnews.com/news/bike-thieves-strike-at-tour-of-slovenia-and-baloise-belgium-tour/
https://www.cyclingnews.com/news/i-wanted-to-make-myself-suffer-chabbey-on-long-range-breakaway-at-tour-de-suisse/
https://www.cyclingnews.com/news/remco-evenepoel-this-was-the-best-way-to-honour-gino/
https://www.cyclingnews.com/news/bahrain-victorious-among-teams-withdrawing-from-final-stages-of-tour-de-suisse/
nan
Error! Next
https://www.cyclingnews.com/news/tour-de-su

In [119]:
all_article_text_df = pd.DataFrame(all_article_text, columns=['url', 'text'])
all_article_text_df.head()

,url,text
0,https://www.cyclingnews.com/news/lorena-wiebes...,Lorena Wiebes (SD Worx) was not on the start l...
1,https://www.cyclingnews.com/news/truck-driver-...,A German court has approved the extradition to...
2,https://www.cyclingnews.com/news/doctors-give-...,Doctors have given Elisa Balsamo the green lig...
3,https://www.cyclingnews.com/news/jumbo-visma-c...,The polemics about contrasting ambitions for W...
4,https://www.cyclingnews.com/news/fabio-jakobse...,Fabio Jakobsen finished last on stage 5 to Lar...


Nun haben wir einen DataFrame mit den Metadaten (url, titel, author etc.) und einen DataFrame mit den jeweiligen Texten und der url. Beide DataFrames können wir nun anhand der url mergen:

In [120]:
all_results_df.dropna(how='all', inplace=True)
all_results_df.head()

,url,title,date,author
0,https://www.cyclingnews.com/news/lorena-wiebes...,Lorena Wiebes leaves Giro Donne to focus on To...,2023-07-06T14:21:02Z,Kirsten Frattini
1,https://www.cyclingnews.com/news/truck-driver-...,Truck driver accused of Davide Rebellin hit-an...,2023-07-06T12:15:38Z,Barry Ryan
3,https://www.cyclingnews.com/news/doctors-give-...,Doctors give Balsamo green light to start trai...,2023-07-06T00:44:18Z,Kirsten Frattini
4,https://www.cyclingnews.com/news/jumbo-visma-c...,Jumbo-Visma combine to outfox Pogacar in Pyren...,2023-07-05T19:11:14Z,Stephen Farrand
5,https://www.cyclingnews.com/news/fabio-jakobse...,Fabio Jakobsen fights the pain to stay in Tour...,2023-07-05T18:31:13Z,Stephen Farrand


In [121]:
cn_df = all_results_df.merge(all_article_text_df, on = 'url', how='left')

In [122]:
cn_df.head()

,url,title,date,author,text
0,https://www.cyclingnews.com/news/lorena-wiebes...,Lorena Wiebes leaves Giro Donne to focus on To...,2023-07-06T14:21:02Z,Kirsten Frattini,Lorena Wiebes (SD Worx) was not on the start l...
1,https://www.cyclingnews.com/news/truck-driver-...,Truck driver accused of Davide Rebellin hit-an...,2023-07-06T12:15:38Z,Barry Ryan,A German court has approved the extradition to...
2,https://www.cyclingnews.com/news/doctors-give-...,Doctors give Balsamo green light to start trai...,2023-07-06T00:44:18Z,Kirsten Frattini,Doctors have given Elisa Balsamo the green lig...
3,https://www.cyclingnews.com/news/jumbo-visma-c...,Jumbo-Visma combine to outfox Pogacar in Pyren...,2023-07-05T19:11:14Z,Stephen Farrand,The polemics about contrasting ambitions for W...
4,https://www.cyclingnews.com/news/fabio-jakobse...,Fabio Jakobsen fights the pain to stay in Tour...,2023-07-05T18:31:13Z,Stephen Farrand,Fabio Jakobsen finished last on stage 5 to Lar...


In [123]:
len(cn_df)

184

In [124]:
# export
cn_df.to_csv('../data/cycling_news_articles.csv', index=False)

**Zeit für Übung 3**

### Ethische und rechtliche Einschränkungen beim Webscrapen

- Auch wenn es möglich ist jeden Part einer Webseite automatisiert zu extrahieren, heißt das NICHT, dass wir das auch dürfen!
- Manche Webseiten verbieten webscraping in ihren Nutzungsbedingungen, und in einigen Fällen können diese Webseits rechtliche Schritte einleiten, wenn wir diese Daten irgendwo verwenden
- Wir wollen auf jeden Fall vermeiden, dass uns beim Web Scraping rechtliche Schritte drohen
- Bitte Prüfen Sie immer die Nutzungsbedingungen der Websites, die Sie scrapen.
- Als allgemeine Regel gilt: Wenn Websites den Zugang zu Daten in großem Umfang ermöglichen wollen, werden sie eine API einrichten, um sie bereitzustellen
- Wenn dies nicht der Fall ist, sollten Sie darauf verzichten, Daten in großem Umfang zu sammeln, und Ihre Datenerfassung auf das beschränken, was Sie benötigen, anstatt alle Daten zu sammeln und später zu filtern
- Manche Websites verweigern den Dienst, wenn wir zu viele Anfragen stellen

Die am weitesten verbreitete Position in der Wissenschaft zu Webscraping scheint zu sein, dass öffentliche Online-Inhalte mit denselben Standards behandelt werden sollten, die man auch bei der Beobachtung von Menschen in öffentlichen Umgebungen anwenden würde, wie es z.B. Ethnographen tun (vgl. McLLevey 2021).